北海道オープンデータプラットフォーム(https://pf.hoda.jp/) で公開されている公共交通GTFSデータ(https://ckan.hoda.jp/dataset/gtfs-data) をGeoJSON形式のファイルに変換します。


In [2]:
import os
import zipfile
import pandas as pd
import json
import io

### 停留所の座標データをGeoJSON (type=Point) に変換

In [3]:
def outputStops(df, fp):
    featureCollection = {
        "type":"FeatureCollection",
        "name":"GTFS stops"
    }
    featureCollection["features"] = []

    for i,s in df.iterrows():
        feature = {
            "type":"Feature"
        }
        feature["geometry"]  = {
            "type":"Point",
            "coordinates":[s['stop_lon'], s['stop_lat']]
        }
        feature["properties"] = {
#             'stop_id':s.stop_id,
#             'stop_name':s.stop_name
        }
        
        for k,v in s.iteritems():
            feature["properties"][k] = v
    
        featureCollection["features"].append(feature)

    json.dump(featureCollection, fp, ensure_ascii=False, indent=1)

### 経路形状のデータをGeoJSON (type=LineString) に変換

In [4]:
def outputShapes(df, fp):
    featureCollection = {
        "type":"FeatureCollection",
        "name":"GTFS shapes"
    }
    featureCollection["features"] = []

    for shpid, df_ in df.groupby('shape_id'):
        feature = {
            "type":"Feature",
            "properties": {
                "shape_id":shpid,
                "lineColor":[80,80,255],
                "lineWidth": 1,
                "opacity":0.8
            },
            "geometry": {
                "type":"LineString",
                "coordinates": df_.loc[:, ['shape_pt_lon', 'shape_pt_lat']].values.tolist()
            }
        }

        featureCollection["features"].append(feature)

    json.dump(featureCollection, fp, ensure_ascii=False, indent=None)

### ZIPファイルからstops.txtを取り出してGeoJSONを出力する

In [5]:
# hodaフォルダ内の.zipファイルをまとめて変換する
for zip_file in filter(lambda f: f[-4:]=='.zip', os.listdir("hoda")):
    bus_name = zip_file.split('.')[0]
    
    path = os.path.join("hoda", zip_file)
    
    with zipfile.ZipFile(path) as _zip:
        for info in _zip.infolist():
            if info.filename=="stops.txt":
                # zipファイルから stops.txt を読み込む
                buffer = io.StringIO(_zip.read(info.filename).decode('utf-8'))
                
                # CSVデータからDataFrameを生成
                df = pd.read_csv(buffer)
                
                # 値が入っていない列は削除
                df = df.dropna(axis='columns', how='all')
                
                print(f"{bus_name}: Num. of stops={df.size}")
                
                with open(os.path.join("GeoJSON", f"{bus_name}_stops.geojson"), "w") as fp:
                    outputStops(df, fp)

abashiri_bus: Num. of stops=2450
akan_bus: Num. of stops=7630
asahikawa_denki: Num. of stops=5712
atsuma_bus: Num. of stops=1834
bankei_bus: Num. of stops=406
dounan_bus: Num. of stops=16730
engan_bus: Num. of stops=5502
eniwashi_com: Num. of stops=973
fukagawashi_com: Num. of stops=511
furano_bus: Num. of stops=1624
haborocho_com: Num. of stops=371
hokkaido_chuo: Num. of stops=36771
joutetsu: Num. of stops=2849
kitami_bus: Num. of stops=9653
kushiro_bus: Num. of stops=7644
meishi_bus: Num. of stops=2884
memurocho_com: Num. of stops=1078
mikasashi_com: Num. of stops=224
naiecho_com: Num. of stops=455
nakashibetsucho_com: Num. of stops=2170
nayoroshi_com: Num. of stops=700
nemuro_kotu: Num. of stops=2380
sharicho_com: Num. of stops=553
shihorocho_com: Num. of stops=420
sorachi_chuo_bus: Num. of stops=3122
syari_bus: Num. of stops=504
takushoku_bus: Num. of stops=4956
tobetsucho_com: Num. of stops=679
tokachi_bus: Num. of stops=11718
yutetsu_bus: Num. of stops=3010
